In [3]:
import numpy as np
import sys

sys.path.append("/home/nilscp/GIT/crater_morphometry")
import geomorphometry
from preprocessing import DEM_extraction
from rim_detection import rim_detection
from pathlib import Path

/home/nilscp/.pyenv/versions/3.8.0/envs/computer-vision/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


## Clip DEM and visible image for rayed craters (SLDEM2013)

In [ ]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya.shp'
dem = "/media/nilscp/pampa/Kaguya/SLDEM2013/SLDEM2013.vrt"
orthoimage = "/media/nilscp/pampa/Kaguya/TCO_MAP_02/TCO_MAP_02.vrt"
clip_distance = 8.0
output_dir = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_RayedCraters/"
output_dir_ortho = "/media/nilscp/pampa/ANALYSIS/2022/TCOMAP_RayedCraters/"
identifier_dem = "Kaguya_SLDEM2013"
identifier_orthoimage = "Kaguya_TCOMAP_02"

In [ ]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance,
                          output_dir, identifier_dem, craterID = None)
                          
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, 
output_dir_ortho, identifier_orthoimage, craterID = None)

## Clip DEM and visible image for coldspots (SLDEM2013)

In [ ]:
location_of_craters = '/home/nilscp/QGIS/Moon/Williams2018/coldspots_larger_than_250m_EQC.shp'
dem = "/media/nilscp/pampa/Kaguya/SLDEM2013/SLDEM2013.vrt"
orthoimage = "/media/nilscp/pampa/Kaguya/TCO_MAP_02/TCO_MAP_02.vrt"
clip_distance = 16.0
output_dir = "/media/nilscp/pampa/ANALYSIS/2022/SLDEM2013_ColdSpots/"
output_dir_ortho = "/media/nilscp/pampa/ANALYSIS/2022/TCOMAP_ColdSpots/"
identifier_dem = "Kaguya_SLDEM2013"
identifier_orthoimage = "Kaguya_TCOMAP_02"

In [ ]:
DEM_extraction.clip_raster_to_crater(location_of_craters, dem, clip_distance,
                          output_dir, identifier_dem, craterID = None)
                          
DEM_extraction.clip_raster_to_crater(location_of_craters, orthoimage, clip_distance, 
output_dir_ortho, identifier_orthoimage, craterID = None)

## Detect rim composite, and update the centre of the crater (Rayed craters)

In [ ]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya.shp'
dem_folder = '/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters/'
shp_folder = '/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/'
scaling_factor = 1.0
suffix = '_Kaguya_SLDEM2013.tif'

In [ ]:
rim_detection.main(location_of_craters, dem_folder, shp_folder,  scaling_factor,
         suffix, craterID = None)

## Detect rim composite, and update the centre of the crater (Coldspots)

In [ ]:
location_of_craters = '/home/nilscp/GIT/crater_morphometry/data/rayed_craters/rayed_craters_centroids_kaguya.shp'
dem_folder = '/home/nilscp/tmp/fresh_impact_craters/SLDEM2015_RayedCraters/'
shp_folder = '/media/nilscp/pampa/ANALYSIS/2022/shapefiles/RayedCraters/'
scaling_factor = 1.0
suffix = '_Kaguya_SLDEM2013.tif'

In [ ]:
rim_detection.main(location_of_craters, dem_folder, shp_folder,  scaling_factor,
         suffix, craterID = None)

## Update centre of craters
- Merge the centres of craters together
- Merge the polygons of rim composite together (for a faster inspection of results). I think I will here 
- If this step is not working properly for a crater, its id is written down (most likely due to too low resolution of the DEM, or very special conditions)

## Update the DEM, and the 2.0R-3.0R and 0.9R-1.1R detrendings
To have the right DEMs

## Second run to detect rim composite, and update the centre of the crater

## Calculate morphometrical parameters from the second ellipse (of the second run)
- load ellipse in pixel coordinates (can also give the possibility to use points and not the ellipse!)
- create an ellipse with 2.0 r
- I could add the possibility to add manually an ellipse for the few cases where the ellipse is not representing well the rim of the crater